In [108]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [109]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward>)


In [110]:
CONTEXT_SIZE = 4
EMBEDDING_SIZE = 10

In [132]:
test_sentences = """「 クリスティーヌ 。 お前 に は 隣国 の 貴族 に 嫁い で もらう 」 「 … … え ？ 」 私 は 驚い て 目 を 見開い た 。 目 の 前 に は 立派 な 口髭 を 生やし た 壮年 の 男 が 、 私 を 見下ろし て いる 。 その 周り に は 、 世 に も 美しい 顔立ち の し た 金髪 碧眼 の 青年 や 、 眼鏡 を 掛け た クール な 美 男子 や 、 その 顔 に あどけな さ を 残す 可愛らしい 美少年 が 、 総じて 私 を 睨ん で い た 。 そして もう 一人 、 素朴 な 可愛らし さ が ある 少女 が 、 困っ た 顔 で 私 を 見 て いる 。 私 は クリスティーヌ ・ ブラン シャーネ 。 王国 有数 の ブラン シャーネ 公爵 の 一人娘 で 、 小さい 頃 から 蝶 よ 花 よ と 育て られ 、 それ は それ は 傲慢 な 娘 に 育っ た 。 生まれ た 時 から 、 望ん だ もの は 全部 与え られる 。 そして それ を 当然 だ と 思っ て い た 。 """.split()
vocab_set = set(test_sentences)
word2id = {word: id for id, word in enumerate(vocab_set)}
id2word = {id: word for word, id in word2id.items()}
data = [
  (
    [word2id[test_sentences[i - 2]], word2id[test_sentences[i - 1]], word2id[test_sentences[i + 1]], word2id[test_sentences[i + 2]]],
    word2id[test_sentences[i]]
  )
  for i in range(CONTEXT_SIZE // 2, len(test_sentences) - CONTEXT_SIZE)
]

In [112]:
vocab_size = len(word2id)

In [118]:
class CBOW(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_dim):
    super(CBOW, self).__init__()
    self.embed = nn.Embedding(vocab_size, embedding_dim)
    self.pre_linear = nn.Linear(CONTEXT_SIZE * embedding_dim, hidden_dim)
    self.suffix_linear = nn.Linear(hidden_dim, vocab_size)
    pass

  def forward(self, x):
    x = self.embed(x).view((1, -1))
    x = self.pre_linear(x)
    x = F.relu(x)
    x = self.suffix_linear(x)
    return F.log_softmax(x, dim=1)


In [123]:
model = CBOW(vocab_size, EMBEDDING_SIZE, 128)
nlloss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
epoch_num = 100

for epoch in range(epoch_num):
  loss_sum = 0
  for contexts, target in data:
    model.zero_grad()
    out = model(torch.tensor(contexts, dtype=torch.long))
    loss = nlloss_function(out, torch.tensor([target], dtype=torch.long))
    loss.backward()
    optimizer.step()
    loss_sum += loss.item()
  if (epoch+1) % 10 == 0:
    print('loss', loss_sum / epoch_num)


CBOW(
  (embed): Embedding(98, 10)
  (pre_linear): Linear(in_features=40, out_features=128, bias=True)
  (suffix_linear): Linear(in_features=128, out_features=98, bias=True)
)
loss 3.312348440140486
loss 0.4157658089697361
loss 0.15780238436069338
loss 0.10380860859295353
loss 0.08053249854361638
loss 0.06709504297585227
loss 0.05828188395360485
loss 0.05193546946858987
loss 0.047075642191339286
loss 0.043335516022052614


In [136]:
parsed = "私 を 見下ろし て いる".split()
data = [word2id[word] for word in parsed]
data = data[:2] + data[3:5]
p = model(torch.tensor(data, dtype=torch.long))

In [137]:
torch.topk(p, 5, largest=False)

torch.return_types.topk(values=tensor([[-12.5611, -12.4337, -12.4280, -12.3653, -12.0035]],
       grad_fn=<TopkBackward>), indices=tensor([[61, 55, 68, 87, 57]]))

In [142]:
print(parsed)
for idx, i in enumerate(torch.topk(p, 5, largest=True).indices[0]):
  print(idx + 1, id2word[i.item()])

['私', 'を', '見下ろし', 'て', 'いる']
1 見
2 見下ろし
3 睨ん
4 驚い
5 私
